## Last Modified: 4/28/21

In [1]:
import pandas as pd
import numpy as np
import nibabel as ni
import scipy.stats as ss
import os

In [2]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

In [4]:
dataset = "acute"

In [5]:
#inp = pd.read_csv(dataset+"_ds.csv")
inp = pd.read_csv(dataset+"_scores.csv")
subjects = inp["subject"].to_list()

In [6]:
cols = ["nih", "lnm_tot", "lnm_pos", "lnm_neg", "vlsm_tot", "vlsm_pos","lnm_tot_avg", "lnm_pos_avg", "lnm_neg_avg", "vlsm_tot_avg", "vlsm_pos_avg", "sp_corr_lnm", "sp_corr_vlsm", "l_size"] # "sp_corr_lnm" and "sp_corr_vlsm" for sp corr vars 
df = pd.DataFrame(index=subjects, columns=cols)

In [5]:
lnm_img = ni.load("template_palm_acute/_vox_tstat.nii")
lnm_ref = np.array(lnm_img.dataobj)
vlsm_img = ni.load("vlsm_acute.nii")
vlsm_ref = np.array(vlsm_img.dataobj)
mask_img = ni.load("mask.nii")
mask_ref = np.array(mask_img.dataobj)
wm_img = ni.load("palm_dcn_acute/_vox_tstat.nii")
wm_ref = np.array(wm_img.dataobj)

In [8]:
vlsm_x = []
lnm_x = []
for x in range(len(vlsm_ref)):
    for y in range(len(vlsm_ref[x])):
        for z in range(len(vlsm_ref[x][y])):
                if mask_ref[x, y, z] > 0:
                    if vlsm_ref[x, y, z] != np.nan:
                        vlsm_x.append(vlsm_ref[x, y, z])
                    if lnm_ref[x, y, z] != np.nan:
                        lnm_x.append(lnm_ref[x, y, z])
vlsm_x = np.array(vlsm_x)
lnm_x = np.array(lnm_x)

### Sums and Avgs

In [9]:
for s in subjects:
    #img = ni.load(inp.loc[inp["subject"] == s, "lesion"].to_list()[0])
    img = ni.load(inp.loc[inp["subject"] == s, "filenames"].to_list()[0])
    data = np.array(img.dataobj)
    lnm_pos = []
    lnm_neg = []
    lnm_tot = []
    vlsm_pos = []
    vlsm_tot = []
    for x in range(len(data)):
        for y in range(len(data[x])):
            for z in range(len(data[x][y])):
                if data[x, y, z] != np.nan and data[x, y, z] > 0:
                    lnm_tot.append(lnm_ref[x, y, z])
                    vlsm_tot.append(vlsm_ref[x, y, z])
                    if lnm_ref[x, y, z] > 0:
                        lnm_pos.append(lnm_ref[x, y, z])
                    if lnm_ref[x, y, z] < 0:
                        lnm_neg.append(lnm_ref[x, y, z])
                    if vlsm_ref[x, y, z] > 0:
                        vlsm_pos.append(vlsm_ref[x, y, z])
    #df.at[s, "nih"] = float(inp.loc[inp["subject"] == s, "nihss_"+dataset].to_list()[0])
    df.at[s, "nih"] = float(inp.loc[inp["subject"] == s, "NIHSS"].to_list()[0])
    df.at[s, "lnm_tot"] = np.sum(np.array(lnm_tot))
    df.at[s, "lnm_pos"] = np.sum(np.array(lnm_pos))
    df.at[s, "lnm_neg"] = np.sum(np.array(lnm_neg))
    df.at[s, "vlsm_tot"] = np.sum(np.array(vlsm_tot))
    df.at[s, "vlsm_pos"] = np.sum(np.array(vlsm_pos))
    if len(lnm_tot) > 0:
        df.at[s, "lnm_tot_avg"] = np.mean(np.array(lnm_tot))
    else:
        df.at[s, "lnm_tot_avg"] = 0
    if len(lnm_pos) > 0:
        df.at[s, "lnm_pos_avg"] = np.mean(np.array(lnm_pos))
    else:
        df.at[s, "lnm_pos_avg"] = 0
    if len(lnm_neg) > 0:
        df.at[s, "lnm_neg_avg"] = np.mean(np.array(lnm_neg))
    else:
        df.at[s, "lnm_neg_avg"] = 0
    if len(vlsm_tot) > 0:
        df.at[s, "vlsm_tot_avg"] = np.mean(np.array(vlsm_tot))
    else:
        df.at[s, "vlsm_tot_avg"] = 0
    if len(vlsm_pos) > 0:
        df.at[s, "vlsm_pos_avg"] = np.mean(np.array(vlsm_pos))
    else:
        df.at[s, "vlsm_pos_avg"] = 0
    df.at[s, "l_size"] = len(lnm_tot)

In [10]:
df["subject"] = subjects

In [10]:
df.to_csv(dataset+"_vars_hospital.csv", index=False)

### Calculate Sp Corr Vars

In [13]:
df = pd.read_csv(dataset+"_vars.csv")
# subjects = list(df["subject"])
subjects = set(int(k) for k in df["subject"])

ValueError: invalid literal for int() with base 10: 'FCS_058'

In [11]:
vlsm_corr = []
lnm_corr = []
#for s in subjects:
for s in range(1, 491):
    if s not in subjects:
        continue
    if s < 10:
        num = "00"+str(s)
    elif s < 100:
        num = "0"+str(s)
    else:
        num = str(s)
    file = ni.load("mgh_avgr_fz/subject"+num+"_AvgR_Fz.nii")
#     try:
#         file = ni.load("brain_lesion_networks/"+s+"_A_lesion_ifhfix_func_seed_T.nii")
#     except:
#         file = ni.load("brain_lesion_networks/"+s+"_A_lesion_func_seed_T.nii")
    nw = np.array(file.dataobj)
    ind_corr = []
    for x in range(len(nw)):
        for y in range(len(nw[x])):
            for z in range(len(nw[x][y])):
                if mask_ref[x, y, z] > 0 and nw[x, y, z] != np.nan:
                    ind_corr.append(nw[x, y, z])
    vlsm_corr.append(ss.pearsonr(vlsm_x, ind_corr)[0])
    lnm_corr.append(ss.pearsonr(lnm_x, ind_corr)[0])
df["sp_corr_vlsm"] = vlsm_corr
df["sp_corr_lnm"] = lnm_corr

In [13]:
df.to_csv(dataset+"_vars_hospital.csv", index=False)

### Calculate White Matter Vars

In [47]:
df = pd.read_csv(dataset+"_vars_cv.csv")

In [48]:
# dmaps = list(pd.read_csv(dataset+"_dcn.csv")["disconnectome"])
dmaps = list(pd.read_csv(dataset+"_palm_dcn.csv")["filenames"])
# lesions = list(pd.read_csv(dataset+"_ds.csv")["lesion"])
lesions = list(pd.read_csv(dataset+"_scores.csv")["filenames"])

In [49]:
wm_total = []
wm_avg = []
# wm_corr = []

In [50]:
for i in range(len(lesions)):
    lesion = np.array(ni.load(lesions[i]).dataobj)
    dmap = np.array(ni.load(dmaps[i]).dataobj)
    wm_val = []
    ind_corr = []
    ref_corr = []
    for x in range(len(lesion)):
        for y in range(len(lesion[x])):
            for z in range(len(lesion[x][y])):
                if lesion[x, y, z] != np.nan and lesion[x, y, z] > 0 and mask_ref[x, y, z] > 0:
                    wm_val.append(wm_ref[x, y, z])
#                 if mask_ref[x, y, z] != np.nan and mask_ref[x, y, z] > 0:
#                     if wm_ref[x, y, z] != np.nan and dmap[x, y, z] != np.nan:
#                         ind_corr.append(dmap[x, y, z])
#                         ref_corr.append(wm_ref[x, y, z])
    wm_total.append(np.sum(np.array(wm_val)))
    wm_avg.append(np.mean(np.array(wm_val)))
#     wm_corr.append(ss.pearsonr(ind_corr, ref_corr)[0])

In [51]:
df["wm_total"] = wm_total
df["wm_avg"] = wm_avg
# df["sp_corr_wm"] = wm_corr

In [53]:
df.to_csv(dataset+"_vars_cv.csv", index=False)

### Seed Region

In [14]:
mask_img = ni.load("mask.nii")
mask_ref = np.array(mask_img.dataobj)
seed_img = ni.load("functional_connectivity_-36_-4_-10.nii")
seed_ref = np.array(seed_img.dataobj)

In [15]:
# inp = pd.read_csv(dataset+"_scores.csv")
inp = pd.read_csv(dataset+"_ds.csv")
subjects = inp["subject"].to_list()
# lesions = list(pd.read_csv(dataset+"_scores.csv")["filenames"])
lesions = list(pd.read_csv(dataset+"_ds.csv")["lesion"])

In [16]:
sr_x = []
for x in range(len(seed_ref)):
    for y in range(len(seed_ref[x])):
        for z in range(len(seed_ref[x, y])):
            if mask_ref[x, y, z] > 0:
                sr_x.append(float(seed_ref[x, y, z]))

In [22]:
sr_df = pd.DataFrame(index=subjects, columns=["sr_total", "sr_pos", "sr_neg", "sr_total_avg", "sr_pos_avg", "sr_neg_avg", "sr_sp_corr"])

In [23]:
# for s in range(1, 491):
for s in range(len(subjects)):
    total = []
    pos = []
    neg = []
    ind_x = []
#     if s not in subjects:
#         continue
#     if s < 10:
#         num = "00"+str(s)
#     elif s < 100:
#         num = "0"+str(s)
#     else:
#         num = str(s)
#     lesion = np.array(ni.load("mgh_lesions/subject"+num+".nii").dataobj)
#     network = np.array(ni.load("mgh_avgr_fz/subject"+num+"_AvgR_Fz.nii").dataobj)
    lesion = np.array(ni.load(lesions[s]).dataobj)
    try:
        network = np.array(ni.load("brain_lesion_networks/"+subjects[s]+"_A_lesion_ifhfix_func_seed_T.nii").dataobj)
    except:
        network = np.array(ni.load("brain_lesion_networks/"+subjects[s]+"_A_lesion_func_seed_T.nii").dataobj)
    for x in range(len(lesion)):
        for y in range(len(lesion[x])):
            for z in range(len(lesion[x, y])):
                if mask_ref[x, y, z] > 0 and lesion[x, y, z] > 0:
                    total.append(seed_ref[x, y, z])
                    if seed_ref[x, y, z] > 0:
                        pos.append(seed_ref[x, y, z])
                    if seed_ref[x, y, z] < 0:
                        neg.append(seed_ref[x, y, z])
                if mask_ref[x, y, z] > 0:
                    ind_x.append(float(network[x, y, z]))
    sr_df.at[subjects[s], "sr_total"] = np.sum(np.array(total))
    sr_df.at[subjects[s], "sr_pos"] = np.sum(np.array(pos))
    sr_df.at[subjects[s], "sr_neg"] = np.sum(np.array(neg))
    if len(total) > 0:
        sr_df.at[subjects[s], "sr_total_avg"] = np.mean(np.array(total))
    else:
        sr_df.at[subjects[s], "sr_total_avg"] = 0
    if len(pos) > 0:
        sr_df.at[subjects[s], "sr_pos_avg"] = np.mean(np.array(pos))
    else:
        sr_df.at[subjects[s], "sr_pos_avg"] = 0
    if len(neg) > 0:
        sr_df.at[subjects[s], "sr_neg_avg"] = np.mean(np.array(neg))
    else:
        sr_df.at[subjects[s], "sr_neg_avg"] = 0
    corr = ss.pearsonr(np.array(sr_x), np.array(ind_x))[0]
    sr_df.at[subjects[s], "sr_sp_corr"] = corr

In [25]:
sr_df.to_csv(dataset+"_seed_region_-36_-4.csv", index=False)